# 연결 설정하기

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

with engine.connect() as conn:
    result = conn.execute(text("SELECT 'hello world'"))
    print(result.all())

2023-12-13 09:11:21,726 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 09:11:21,727 INFO sqlalchemy.engine.Engine SELECT 'hello world'
2023-12-13 09:11:21,728 INFO sqlalchemy.engine.Engine [generated in 0.00280s] ()
[('hello world',)]
2023-12-13 09:11:21,730 INFO sqlalchemy.engine.Engine ROLLBACK


# 변경 사항 커밋하기
### Commit as you go

In [2]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [
            {"x": 1, "y": 1},
            {"x": 2, "y": 4}
        ]
    )
    conn.commit()

2023-12-13 09:11:21,744 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 09:11:21,746 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-12-13 09:11:21,747 INFO sqlalchemy.engine.Engine [generated in 0.00334s] ()
2023-12-13 09:11:21,751 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 09:11:21,752 INFO sqlalchemy.engine.Engine [generated in 0.00170s] [(1, 1), (2, 4)]
2023-12-13 09:11:21,753 INFO sqlalchemy.engine.Engine COMMIT


### Begin once

In [3]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [
            {"x": 6, "y": 8},
            {"x": 9, "y": 10}
        ]
    )

2023-12-13 09:11:21,769 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 09:11:21,771 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 09:11:21,771 INFO sqlalchemy.engine.Engine [cached since 0.0207s ago] [(6, 8), (9, 10)]
2023-12-13 09:11:21,772 INFO sqlalchemy.engine.Engine COMMIT


# SQL 문 실행하기
## 행 가져오기

In [4]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x} y: {row.y}")

2023-12-13 09:11:21,787 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 09:11:21,788 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 09:11:21,789 INFO sqlalchemy.engine.Engine [generated in 0.00236s] ()
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2023-12-13 09:11:21,791 INFO sqlalchemy.engine.Engine ROLLBACK


여러 변수에 할당하여 행에 접근하기

In [5]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for x, y in result:
        print(f"x: {x} y: {y}")

2023-12-13 10:17:08,734 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 10:17:08,736 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 10:17:08,737 INFO sqlalchemy.engine.Engine [cached since 3947s ago] ()
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2023-12-13 10:17:08,740 INFO sqlalchemy.engine.Engine ROLLBACK


정수 인덱스로 행에 접근하기

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        x = row[0]
        y = row[1]
        print(f"x: {x} y: {y}")

2023-12-13 10:21:52,370 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 10:21:52,371 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 10:21:52,371 INFO sqlalchemy.engine.Engine [cached since 4231s ago] ()
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2023-12-13 10:21:52,372 INFO sqlalchemy.engine.Engine ROLLBACK


속성 이름으로 행에 접근하기

In [7]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        x = row.x
        y = row.y
        print(f"x: {x} y: {y}")

2023-12-13 10:23:08,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 10:23:08,964 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 10:23:08,965 INFO sqlalchemy.engine.Engine [cached since 4307s ago] ()
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2023-12-13 10:23:08,966 INFO sqlalchemy.engine.Engine ROLLBACK


매핑 객체로 행에 접근하기

In [9]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]
        print(f"x: {x} y: {y}")

2023-12-13 10:23:59,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 10:23:59,124 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 10:23:59,125 INFO sqlalchemy.engine.Engine [cached since 4357s ago] ()
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2023-12-13 10:23:59,127 INFO sqlalchemy.engine.Engine ROLLBACK


매개 변수 전달하기

In [10]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for x, y in result:
        print(f"x: {x} y: {y}")

2023-12-13 11:08:05,346 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 11:08:05,347 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2023-12-13 11:08:05,347 INFO sqlalchemy.engine.Engine [generated in 0.00172s] (2,)
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2023-12-13 11:08:05,348 INFO sqlalchemy.engine.Engine ROLLBACK


여러 매개 변수 전달하기

In [11]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [
            {"x": 11, "y": 12},
            {"x": 13, "y": 14}
        ]
    )
    conn.commit()

2023-12-13 13:33:56,173 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 13:33:56,175 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 13:33:56,176 INFO sqlalchemy.engine.Engine [cached since 1.575e+04s ago] [(11, 12), (13, 14)]
2023-12-13 13:33:56,177 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
from sqlalchemy.orm import Session


stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")

with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x} y: {row.y}")

2023-12-13 16:06:08,023 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:06:08,026 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2023-12-13 16:06:08,027 INFO sqlalchemy.engine.Engine [generated in 0.00134s] (6,)
x: 6 y: 8
x: 9 y: 10
x: 11 y: 12
x: 13 y: 14
2023-12-13 16:06:08,029 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y = :y WHERE x = :x"),
        [
            {"x": 9, "y": 11},
            {"x": 13, "y": 15}
        ]
    )
    session.commit()

2023-12-13 16:20:15,939 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:20:15,941 INFO sqlalchemy.engine.Engine UPDATE some_table SET y = ? WHERE x = ?
2023-12-13 16:20:15,942 INFO sqlalchemy.engine.Engine [generated in 0.00117s] [(11, 9), (15, 13)]
2023-12-13 16:20:15,943 INFO sqlalchemy.engine.Engine COMMIT
